# Day 3 - Lab 1: AI-Driven Backend Development

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 3! With our requirements and architecture defined, it's time to write code. In this lab, you will act as a senior developer guiding an AI co-pilot. Your task is to generate the full backend API for the Onboarding Tool. This involves not just generating code, but also connecting it to the live database we created yesterday, moving from a prototype to a functional, data-driven application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll set up our environment and load the `schema.sql` artifact from Day 2. This SQL file contains the `CREATE TABLE` statements that define our database structure, which is the perfect context to provide the LLM for code generation.

**Model Selection:**
For code generation, models specifically fine-tuned for coding are ideal. `gpt-4.1`, `o3`, or `codex-mini` are excellent choices. Experiment to see which one gives you the cleanest code.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the SQL schema.
- `save_artifact()`: To save the generated Python code.
- `clean_llm_output()`: To remove markdown fences from the generated code.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/schema.sql")
if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges

Follow the challenges below to build and connect your API.

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Task:** Generate all the necessary Python code for a FastAPI application, but with simple in-memory data storage for now. This allows us to generate and validate the code's structure before adding database complexity.

**Instructions:**
1.  Create a detailed prompt that asks the LLM to act as a senior Python developer.
2.  Provide the `sql_schema` as context.
3.  Instruct the LLM to generate three key components:
    * **Pydantic Models:** For API data validation (request/response bodies).
    * **FastAPI Endpoints:** Full CRUD (Create, Read, Update, Delete) endpoints for the `users` table.
    * **In-Memory Database:** A simple Python list to act as a temporary, fake database.
4.  The final output should be a single Python script for a `main_in_memory.py` file.
5.  Save the generated code to `app/main_in_memory.py`.

In [2]:
# TODO: Write a prompt to generate a complete FastAPI application with in-memory data storage.
in_memory_api_prompt = f"""
You are a senior Python developer tasked with creating a FastAPI application for a new hire onboarding tool. 
The application will serve as a prototype to demonstrate the API structure and functionality.

**Requirements:**
Based on the following SQL schema, generate a single Python script for a `main.py` file that includes:
1. **Imports and Setup**:
   - All necessary FastAPI imports.
   - Include `pydantic` for data validation and `typing` for type annotations.
   - Add comments and docstrings to explain the purpose of each section and function.

2. **Pydantic Models**:
   - Define models for creating and reading `User` resources.
   - Include fields: `id` (int), `name` (str), `email` (str), and `role` (str).
   - Add validation rules for `email` (must be a valid email) and `role` (must be one of 'New Hire', 'HR Manager', 'Department Manager').

3. **In-Memory Database**:
   - Use a Python list to act as a fake database for storing user data.
   - Include sample data for testing purposes.

4. **CRUD Endpoints**:
   - Implement endpoints for the `/users` path:
     - **POST**: Create a new user. Validate input and ensure `email` is unique.
     - **GET (all)**: Retrieve all users with optional query parameters for filtering by `role`.
     - **GET (by ID)**: Retrieve a user by their `id`. Return a 404 error if the user is not found.
     - **PUT**: Update an existing user by `id`. Validate input and handle cases where the user does not exist.
     - **DELETE**: Delete a user by `id`. Return a 404 error if the user is not found.

5. **Error Handling**:
   - Include appropriate HTTP status codes and error messages for invalid requests.
   - Use FastAPI's `HTTPException` for error handling.

6. **Output Expectations**:
   - Ensure the code is formatted and adheres to PEP 8 standards.
   - Include a `__main__` block to run the FastAPI application.

**SQL Schema Context:**
```sql
{sql_schema}
```

Output only the raw Python code.
"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/main_in_memory.py")
    print("Saved in-memory API to app/main_in_memory.py")
else:
    print("Skipping API generation because schema is missing.")

--- Generating FastAPI app with in-memory database ---
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, EmailStr, validator
from typing import List, Optional
from fastapi.middleware.cors import CORSMiddleware

# Initialize FastAPI application
app = FastAPI()

# Allow all CORS origins for demonstration purposes
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Pydantic models for input validation and data representation
class UserBase(BaseModel):
    name: str
    email: EmailStr
    role: str

    @validator('role')
    def validate_role(cls, v):
        valid_roles = ['New Hire', 'HR Manager', 'Department Manager']
        if v not in valid_roles:
            raise ValueError(f'Role must be one of {valid_roles}')
        return v

class UserCreate(UserBase):
    pass

class UserRead(UserBase):
    id: int

# In-memory "database" of users
fake_db: List[UserRead] = [
    UserRead(id=1, name='

### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Task:** Now, generate the specific SQLAlchemy code required to connect our application to the live `onboarding.db` SQLite database.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `sql_schema` as context again.
3.  Instruct the LLM to generate two separate pieces of code:
    * **SQLAlchemy Models:** Python classes that map to your database tables.
    * **Database Session Management:** The boilerplate code to create a database engine, session maker, and a dependency function (`get_db`) for use in FastAPI.
4.  The output should be two distinct, well-commented Python code blocks. We will integrate these manually in the next step.

In [4]:
# TODO: Write a prompt to generate SQLAlchemy models and the database session/dependency code.
db_code_prompt = f"""
ROLE: You are a Python expert specializing in FastAPI and SQLAlchemy.
TASK: Generate Python code that connects a FastAPI application to a SQLite database named 'onboarding.db'. The code should follow best practices for maintainability, scalability, and readability.

**Requirements:**
Based on the provided SQL schema, generate two separate, well-commented code blocks:

1. **SQLAlchemy Models:**
   - Create Python classes that map to the `users` and `onboarding_tasks` tables.
   - Include appropriate data types, constraints, and relationships as defined in the schema.
   - Use SQLAlchemy's `relationship()` to establish the one-to-many relationship between `users` and `onboarding_tasks`.
   - Add comments explaining the purpose of each class and its attributes.

2. **Database Session Management:**
   - Provide the boilerplate code for:
     - Creating the SQLAlchemy engine with the SQLite database (`onboarding.db`).
     - Defining the `SessionLocal` class for managing database sessions.
     - Implementing the `get_db` dependency for FastAPI to ensure proper session lifecycle management.
   - Include comments explaining each step and its role in the database connection process.

3. **Additional Requirements:**
   - Ensure the code adheres to PEP 8 standards.
   - Include a `Base.metadata.create_all(bind=engine)` call to create the database tables if they do not already exist.
   - Use `connect_args={{"check_same_thread": False}}` for SQLite to avoid threading issues.
   - Add error handling for database connection issues.

**SQL Schema Context:**
SQL:
{sql_schema}

Output only the raw Python code.
"""

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models and Session Code ---

--- Generated Database Code ---
```python
# models.py

from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Text, CheckConstraint
from sqlalchemy.orm import relationship, declarative_base

# Create a base class for declarative models
Base = declarative_base()

class User(Base):
    """
    SQLAlchemy model for the 'users' table.
    
    Attributes:
        id (int): Primary key, auto-incremented.
        name (str): Full name of the user, cannot be empty.
        email (str): Unique email address of the user, used for login and notifications.
        role (str): Role of the user within the system, with constraints on possible values.
    """
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)
    email = Column(String, unique=True, nullable=False)
    role = Column(String, nullable=False, 
                  CheckConstraint("role

### Challenge 3 (Advanced): Integrating Live Database Logic

**Task:** This is the most critical engineering step of the lab. You will manually integrate the generated database code into the FastAPI application, replacing the in-memory logic with live database operations.

**Instructions:**
This task represents a significant jump in complexity. Follow these steps carefully in your IDE (like VS Code):

1.  Create a new, empty file named `app/main.py`.
2.  **First, copy the Pydantic models and the `app = FastAPI()` line** from your `app/main_in_memory.py` file and paste them into `app/main.py`.
3.  **Next, paste the SQLAlchemy model classes and the `get_db` dependency function** you generated in Challenge 2 into your new `app/main.py`.
4.  **Now, let's refactor the `POST /users/` endpoint.** Copy the endpoint function from the in-memory file, but replace the in-memory logic (e.g., `db.append()`) with the correct SQLAlchemy session calls: `db.add(db_user)`, `db.commit()`, and `db.refresh(db_user)`.
5.  Repeat this refactoring process for the other endpoints (GET, PUT, DELETE), replacing list manipulations with the appropriate SQLAlchemy `db.query()` methods.

This task requires you to act as the senior developer, stitching together the AI-generated components into a functional, cohesive whole. You may need to ask the LLM follow-up questions like, "How do I write a SQLAlchemy query to find a user by ID?"

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.